<a href="https://colab.research.google.com/github/Edwin1719/GPT4o-Web-Agent-Serper/blob/main/Tool_calling_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![texto del vínculo](https://i.ytimg.com/vi/a_YC8Ru6Jd0/maxresdefault.jpg)

In [32]:
# Instalacion de Recursos y dependencias
!pip install openai requests langchain langchainhub langchain_community -q

In [33]:
# === Importacion de Librerías necesarias ===
from openai import OpenAI
import requests
from difflib import SequenceMatcher
from google.colab import userdata
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

# === TU CONFIGURACIÓN API_KEY y MODELO LLM ===
API_KEY_OPENAI = userdata.get('API_KEY_OPENAI')
SERPER_API_KEY = userdata.get('SERPER_API_KEY')
MODEL = "gpt-4o"  # o "gpt-3.5-turbo"

In [31]:
# === Cliente LLM de OpenAI ===
llm = ChatOpenAI(
    openai_api_key=API_KEY_OPENAI,
    temperature=0.6,
    model_name="gpt-4o"
)

# === Función para usar Serper.dev ===
def search_web(query: str) -> str:
    headers = {
        "X-API-KEY": SERPER_API_KEY,
        "Content-Type": "application/json"
    }
    data = {"q": query}
    try:
        res = requests.post("https://google.serper.dev/search", headers=headers, json=data)
        result = res.json()
        if "organic" in result and result["organic"]:
            top = result["organic"][0]
            return f"{top['title']}\n{top['snippet']}\n🔗 {top['link']}"
        return "No se encontró información relevante en la web."
    except Exception as e:
        return f"Error al consultar Serper.dev: {e}"

# === Herramienta personalizada para LangChain ===
herramienta_web = Tool(
    name="Buscador Serper",
    func=search_web,
    description="Utiliza esta herramienta para buscar información actualizada en la web cuando no tengas la respuesta."
)

# === Inicialización del agente con herramientas ===
agent = initialize_agent(
    tools=[herramienta_web],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# === Bucle interactivo ===
if __name__ == "__main__":
    print("🤖 Agente GPT-4 con acceso a búsqueda web (Serper.dev)")
    while True:
        query = input("\n❓ Tu pregunta ('salir' para terminar): ")
        if query.lower() in ["salir", "exit", "quit"]:
            break
        respuesta = agent.run(query)
        print(f"\n💬 Respuesta:\n{respuesta}")

🤖 Agente GPT-4 con acceso a búsqueda web (Serper.dev)

❓ Tu pregunta ('salir' para terminar): Ultimas noticias del senado en colombia


> Entering new AgentExecutor chain...
I should use Buscador Serper to find the latest news from the Colombian Senate.
Action: Buscador Serper
Action Input: "Últimas noticias del Senado en Colombia"
Observation: Noticias - Senado
Proyecto para proteger y restaurar las costas avanza en Comisión Quinta del Senado · Noticias 04 Abril 2025 ... Colombia, que hoy se encuentran en ...
🔗 https://www.senado.gov.co/index.php/el-senado/noticias
Thought:I should check the official website of the Colombian Senate for more news updates.
Action: Buscador Serper
Action Input: "Senado de Colombia noticias"
Observation: Noticias - Senado
Proyecto para proteger y restaurar las costas avanza en Comisión Quinta del Senado · Noticias 04 Abril 2025 ... En Colombia estamos muy expuestos ...
🔗 https://www.senado.gov.co/index.php/el-senado/noticias
Thought:I have gathered enough